In [7]:
# LangChain Implementation with TogetherAI using LLM Chain with Chain of Thought (CoT) and Few-Shot Learning
import os
import together
from pathlib import Path
from pypdf import PdfReader
from typing import List, Dict
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.llms.base import LLM
from pydantic import BaseModel, Field
from __future__ import annotations

In [8]:
# Prompt Template with Few‑Shot Learning and Validation/Refinement Pipeline
"""
This module implements a complete question‑answering pipeline that
  1. Builds a few‑shot prompt from simple Q‑A pairs that come only from the
     reference PDF (the impact of artificial intelligence – an economic analysis).
  2. Generates an answer that must stay grounded in that reference.
  3. Runs a validator chain to label each claim as SUPPORTED / UNSUPPORTED / PARTIALLY SUPPORTED.
  4. Iteratively refines the answer (up to 5 passes) until no unsupported claims remain.
  5. If unsupported claims persist after 5 iterations, the best attempt is returned along with
     a message stating that the maximum refinement depth has been reached.
The code is based on LangChain and uses Together AI as the inference provider
"""

# LLM SET‑UP -----------------------------------------------------------
# Setup TogetherAI client

# modify TogetherAI Free API Key as required below
api_key = "e5cb3bf0234845ccad2fdd134cd4224691d2e0da722818509d301fdaf36afc6b"
client = together.Together(api_key=api_key)
MODEL_NAME = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

In [9]:
# Custom LLM Class to Integrate TogetherAI with LangChain
class TogetherAI_LLM(LLM):

    # define model name as a pydantic field
    model_name: str = Field(default=MODEL_NAME)

    # method for handling sending prompts to TogetherAI LLM & retrieving responses
    def _call(self, prompt: str, stop: List[str] = None) -> str:
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2000
        )
        return response.choices[0].message.content

    # identifies the LLM type as together-ai
    @property
    def _llm_type(self) -> str:
        return "together-ai"

# Create an instance of TogetherAI LLM class for use in prompt chains
together_llm = TogetherAI_LLM()

In [10]:
# Function to extract text from PDF
def get_pdf_text(file_path: str) -> str:
    text = ""
    try:
        with Path(file_path).open("rb") as f:
            reader = PdfReader(f)
            text = "\n\n".join([page.extract_text() for page in reader.pages])
    except Exception as e:
        raise Exception(f"Error reading the PDF file: {str(e)}")

    if len(text) > 400_000:
        raise Exception("PDF too long. Limit to 131072 characters.")
    return text

In [11]:
# Few‑shot examples (not one of the evaluation prompts )
# All examples are deliberately *simple* factual snippets lifted from the
# reference PDF so that the LLM gets a clear style/format to imitate.
example_prompt = PromptTemplate.from_template(
    "Question: {question}\nAnswer: {answer}"
)

examples: List[Dict[str, str]] = [
    {
        "question": "What is 'Artificial Intelligence' from an economic perspective?",
        "answer": (
            "The reference document (the impact of artificial intelligence – an economic analysis) defines AI as the capability of "
            "machines to perform tasks that would typically require a human, like reasoning or problem solving, learning, and perception. "
        ),
    },
    {
        "question": "What is one of the factors identified as contributing to New Zealand's slow diffusion of new technology?",
        "answer": (
            "Inflexible or outdated regulations: regulatory systems that have been cited as problematic include data access "
            "competition policy, genetic modification controls, and land use policy (New Zealand Productivity Commission, 2020). "
            "as stated by the reference document. "
        ),
    },
    {
        "question": "What is 'so-so automation'?",
        "answer": (
            "'so-so automation' refers to automation that reduces wages without significant productivity gains "
            "- self-checkouts in grocery stores being one example. "
        ),
    },
]

In [12]:
# Initial few-shot prompt ---------------------------------------------
prefix = (
    "You are an assistant with access to the following reference document.\n"
    "Answer ONLY from this reference – cite nothing else and do not guess.\n\n"
    "Reference (verbatim extract):\n{reference}\n\n"  # passed in at runtime
)

suffix = (
    "\nQuestion: {prompt}\nAnswer:"  # {prompt} supplied at runtime
)

# Define the initial few-shot prompt template used to generate the first LLM response.
# - prefix: introductory instructions and the reference document content.
# - suffix: appends the user’s question after the few-shot examples.
# - examples: a list of simple question-answer pairs extracted from the reference document.
# - example_prompt: the format for rendering each example.
# - input_variables: specifies which fields are dynamically passed in at runtime.
initial_prompt_template: FewShotPromptTemplate = FewShotPromptTemplate(
    prefix=prefix,
    suffix=suffix,
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["reference", "prompt"],
)

In [13]:
# Validation & refinement -----------------------------------
validation_prompt_template = PromptTemplate(
    input_variables=["reference", "response"],
    template=(
        "You are a fact‑checking assistant. For EACH factual claim in the response, "
        "state whether it is SUPPORTED, UNSUPPORTED, or PARTIALLY SUPPORTED by "
        "the reference.\n\nReference:\n{reference}\n\nResponse:\n{response}\n\n"
        "Return a bullet list of labelled claims followed by an overall verdict."
    ),
)

refinement_prompt_template = PromptTemplate(
    input_variables=["reference", "response"],
    template=(
        "The previous answer contained unsupported claims. Using ONLY the reference "
        "below, rewrite the answer so that every statement is fully supported. "
        "If a point cannot be supported, omit it.\n\nReference:\n{reference}\n\n"
        "Current answer:\n{response}\n\nRevised answer:"),
)

In [14]:
# Executes a LangChain prompt by piping the prompt template into the TogetherAI LLM.
# Returns the generated response based on the provided inputs.
def run_llm_chain(prompt_template, inputs: dict) -> str:
    runnable = prompt_template | together_llm
    return runnable.invoke(inputs)

In [15]:
def iterative_refinement_chain(reference: str, draft_answer: str) -> str:
    max_iterations = 5
    for i in range(max_iterations):
        validation_result = run_llm_chain(
            validation_prompt_template,
            {"reference": reference, "response": draft_answer},
        )
        print(f"\n[Iteration {i+1} Validation Result]:\n{validation_result}\n")

        if "UNSUPPORTED" not in validation_result and "PARTIALLY SUPPORTED" not in validation_result:
            print("All claims supported. No refinement needed.")
            return draft_answer

        print("Unsupported claims found. Refining response...")
        draft_answer = run_llm_chain(
            refinement_prompt_template,
            {"reference": reference, "response": draft_answer},
        )
    print("Maximum iterations reached. Returning best attempt.")
    return draft_answer

In [16]:
# Execution pipeline ------------------------------------------------------------
def run_pipeline(reference_path: str | Path, prompts: List[str]):
    reference_content = get_pdf_text(reference_path)

    for i, prompt in enumerate(prompts, start=1):
        print(f"\n=== Prompt {i} ===\n{prompt}\n")
        initial_answer = run_llm_chain(
            initial_prompt_template,
            {"reference": reference_content, "prompt": prompt},
        )
        final_answer = iterative_refinement_chain(reference_content, initial_answer)
        print(f"Final Answer {i}:\n{final_answer}\n")

# default LLM response without in-context learning w.r.t reference document
def run_llm_without_reference(prompt: str) -> str:
    return together_llm.invoke(prompt)

In [17]:
if __name__ == "__main__":
    test_prompt = "What is the predicted global economic impact of AI on employment?"
    print(f"Default Response without Reference:\n{run_llm_without_reference(test_prompt)}")

    reference_path = "impact-of-ai-an-economic-analysis.pdf"
    test_prompts = [
        "What is the predicted global economic impact of AI on employment?",
        "How will AI influence wage inequality globally?",
        "What regulatory challenges are anticipated with AI adoption?",
        "How will AI adoption differ across regions and countries?",
        "What industries are most likely to be disrupted by AI according to the report?"
    ]
    run_pipeline(reference_path, test_prompts)

Default Response without Reference:
The predicted global economic impact of AI on employment is a complex and multifaceted topic. While AI has the potential to bring significant benefits, such as increased productivity and efficiency, it also poses significant challenges for workers and the labor market. Here are some predictions and estimates:

1. **Job displacement**: According to a report by the McKinsey Global Institute, up to 800 million jobs could be lost worldwide due to automation by 2030. Another report by the World Economic Forum estimates that by 2022, more than a third of the desired skills for most jobs will be comprised of skills that are not yet considered crucial to the job today.
2. **Job creation**: However, while AI may displace some jobs, it is also expected to create new ones. A report by the World Economic Forum estimates that by 2022, 75 million jobs may be displaced by automation, but 133 million new jobs may be created.
3. **Net job loss**: Despite the creation